In [1]:
# encoding = utf8
'''
    @Author: King
    @Date: 2019.05.16
    @Purpose: EDA：最简单的自然语言处理数据增广方法
    @Introduction:  EDA：最简单的自然语言处理数据增广方法
    @Datasets: Chinese relation extration datasets
    @Link : https://mp.weixin.qq.com/s?__biz=MzAxMTU5Njg4NQ==&mid=100001705&idx=5&sn=eaa4b3a5a911f7f4068eff98ab7c7867
    @Reference : https://github.com/jasonwei20/eda_nlp
    @paper ： 
'''

'\n    @Author: King\n    @Date: 2019.05.16\n    @Purpose: EDA：最简单的自然语言处理数据增广方法\n    @Introduction:  EDA：最简单的自然语言处理数据增广方法\n    @Datasets: Chinese relation extration datasets\n    @Link : https://mp.weixin.qq.com/s?__biz=MzAxMTU5Njg4NQ==&mid=100001705&idx=5&sn=eaa4b3a5a911f7f4068eff98ab7c7867\n    @Reference : https://github.com/jasonwei20/eda_nlp\n    @paper ： \n'

## EDA：最简单的自然语言处理数据增广方法

我向你介绍EDA：简单数据增广技术，可以大大提升文本分类任务的性能（在EDA Github repository有简单的实现代码）。EDA包含四个简单操作，能极好地防止过拟合，并训练出更强健的模型，分别是：

1. 同义词替换：在句子中随机选取n个非停用词。对每个选取的词，用它的随机选取的同义词替换。

2. 随机插入：在句子中任意找一个非停用词，随机选一个它的同义词，插入句子中的任意位置。重复n次。

3. 随机交换：任意选取句子中的两个词，交换位置。重复n次。

4. 随机删除：对于句子中概率为p的每一个词，随机删除。

这些技术真有效吗？出乎意料，答案是肯定的。尽管生成的某些句子有点怪异，但是在数据集中的引入一些噪声，对于训练出一个健壮的模型来说，是极有好处的，特别是数据集比较小的时候。

### 一、随机采样

#### 1、引入包

In [1]:
import pandas as pd 
import numpy as np

In [8]:
# 功能：内容信息查询方法
def describe_df(df):
    '''
	功能：df 内容信息查询方法
	:param x:    需要查询信息的变量df
    :return:  
	'''
    #print("Type: {0}".format(type(df)))
    print("Shape/size: {0}".format(len(df)))
    print("df.iloc[0]:{0}".format(df.iloc[0])) 

#### 2、数据加载

In [9]:
'''
    quoting : int or csv.QUOTE_* instance, default 0
        控制csv中的引号常量。可选 QUOTE_MINIMAL (0), QUOTE_ALL (1), QUOTE_NONNUMERIC (2) or QUOTE_NONE (3)
    参考地址：
        https://www.cnblogs.com/datablog/p/6127000.html
        https://blog.csdn.net/longwei92/article/details/84326138
'''
type = 'train'
base_path = 'E:/pythonWp/game/CCKS2019/RelationshipExtraction/origin_data/'
df = pd.read_csv("{0}{1}".format(base_path,"train_char.txt"),quoting = 3,sep='\t',names=['e1','e2','r','s'])
describe_df(df)

Shape/size: 287351
df.iloc[0]:e1                           金泰熙
e2                            金东
r                            NaN
s     韩国梦想演唱会第十届2004年:MC:金泰熙，金东万
Name: 0, dtype: object


#### 3、将 nan 替换为 NA 字符 

In [10]:
# 将 nan 替换为 NA 字符 
df_all = df.fillna("NA")
describe_df(df_all)
# pandas统计个数 参考：https://blog.csdn.net/waple_0820/article/details/80514073
df_all['r'].value_counts()

Shape/size: 287351
df.iloc[0]:e1                           金泰熙
e2                            金东
r                             NA
s     韩国梦想演唱会第十届2004年:MC:金泰熙，金东万
Name: 0, dtype: object


NA                               248850
人物关系/亲属关系/配偶/丈夫/现夫                 8135
人物关系/亲属关系/血亲/自然血亲/父母/生父            6870
人物关系/亲属关系/配偶/妻子/现妻                 5513
人物关系/师生关系/老师                       2900
人物关系/亲属关系/血亲/自然血亲/子女/儿子            2627
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/哥哥          1673
人物关系/社交关系/友谊关系/朋友                  1610
人物关系/亲属关系/血亲/自然血亲/父母/生母            1383
人物关系/社交关系/感情关系/喜欢                  1301
人物关系/社交关系/感情关系/恋人                  1266
人物关系/亲属关系/血亲/自然血亲/子女/女儿             830
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/妹妹           805
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/弟弟           637
人物关系/师生关系/学生                        547
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/姐姐           532
人物关系/亲属关系/血亲/自然血亲/祖父母/爷爷            291
人物关系/亲属关系/配偶/妻子/前妻                  245
人物关系/亲属关系/配偶/丈夫/前夫                  218
人物关系/亲属关系/配偶/丈夫/未婚夫                 183
人物关系/亲属关系/姻亲/配偶的血亲/配偶的父母/岳父         165
人物关系/亲属关系/血亲/自然血亲/侄甥/侄子             158
人物关系/亲属关系/姻亲/血亲的配偶/子女的配偶/女婿         119
人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/舅舅           77
人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/叔伯           77


#### 4、清除 NA 关系

In [12]:
# 清除 NA 关系
df_rm_NA = df.dropna() 
describe_df(df_rm_NA)
# pandas统计个数 参考：https://blog.csdn.net/waple_0820/article/details/80514073
df_rm_NA['r'].value_counts()

Shape/size: 38501
df.iloc[0]:e1                          窦滔
e2                          苏蕙
r           人物关系/亲属关系/配偶/妻子/现妻
s     苏蕙思念不已，织绵为《回文璇玑图诗》，寄赠窦滔。
Name: 248850, dtype: object


人物关系/亲属关系/配偶/丈夫/现夫               8135
人物关系/亲属关系/血亲/自然血亲/父母/生父          6870
人物关系/亲属关系/配偶/妻子/现妻               5513
人物关系/师生关系/老师                     2900
人物关系/亲属关系/血亲/自然血亲/子女/儿子          2627
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/哥哥        1673
人物关系/社交关系/友谊关系/朋友                1610
人物关系/亲属关系/血亲/自然血亲/父母/生母          1383
人物关系/社交关系/感情关系/喜欢                1301
人物关系/社交关系/感情关系/恋人                1266
人物关系/亲属关系/血亲/自然血亲/子女/女儿           830
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/妹妹         805
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/弟弟         637
人物关系/师生关系/学生                      547
人物关系/亲属关系/血亲/自然血亲/兄弟姐妹/姐姐         532
人物关系/亲属关系/血亲/自然血亲/祖父母/爷爷          291
人物关系/亲属关系/配偶/妻子/前妻                245
人物关系/亲属关系/配偶/丈夫/前夫                218
人物关系/亲属关系/配偶/丈夫/未婚夫               183
人物关系/亲属关系/姻亲/配偶的血亲/配偶的父母/岳父       165
人物关系/亲属关系/血亲/自然血亲/侄甥/侄子           158
人物关系/亲属关系/姻亲/血亲的配偶/子女的配偶/女婿       119
人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/舅舅         77
人物关系/亲属关系/血亲/自然血亲/叔伯姑舅姨/叔伯         77
人物关系/亲属关系/配偶/妻子/未婚妻                69
人物关系/亲属关系/姻亲/血亲的配偶/兄弟姐妹的配偶/嫂子      67
人物关系/亲属关系/血亲

#### 5、获得 NA 关系数据

In [13]:
# 获得 NA 关系数据
df_na = df[df['r'].isnull().values==True]
describe_df(df_na)

Shape/size: 248850
df.iloc[0]:e1                           金泰熙
e2                            金东
r                            NaN
s     韩国梦想演唱会第十届2004年:MC:金泰熙，金东万
Name: 0, dtype: object


#### 6、对 NA 关系数据进行随机抽样

In [14]:
# 对 NA 关系数据进行 10% 随机抽样
# 参考：python-Pandas学习 如何对数据集随机抽样？ https://blog.csdn.net/qq_22238533/article/details/71080942
simple_rate =0.1
df_na_sample = df_na.sample(n=int(len(df_na)*simple_rate))
describe_df(df_na_sample)

Shape/size: 24885
df.iloc[0]:e1                                孔子
e2                               匡亚明
r                                NaN
s     匡亚明和他的《孔子评传》人民日报（海外版）1985年9月8日
Name: 169508, dtype: object


#### 7、对 抽样  的Na 数据 与 非 NA 的数据合并

In [15]:
# 对 抽样  的Na 数据 与 非 NA 的数据合并
# 合并非空的列表 test_info_vec_df_rm_NA、随机抽样后的空列表 test_info_vec_df_na_sample
# 现将表构成list，然后在作为concat的输入
# 参考：PANDAS 数据合并与重塑（concat篇）: https://blog.csdn.net/stevenkwong/article/details/52528616
frames = [df_rm_NA,df_na_sample]
new_sample_df = pd.concat(frames)
describe_df(new_sample_df)
#new_test_df.to_csv("sample50_test.txt",index=None,header=None)

Shape/size: 63386
df.iloc[0]:e1                          窦滔
e2                          苏蕙
r           人物关系/亲属关系/配偶/妻子/现妻
s     苏蕙思念不已，织绵为《回文璇玑图诗》，寄赠窦滔。
Name: 248850, dtype: object


#### 8、采样数据保存

In [16]:
from sklearn.utils import shuffle
new_sample_df = shuffle(new_sample_df)

new_sample_df.to_csv("sample{0}_{1}_sent.txt".format(str(int(simple_rate*100)),type),index=None,header=None,sep='\t')